In [14]:
#Q1: Does patient gender predict APACHE score? (SE)
#Q1A: Does patient gender predict APACHE IV score? (SE)
#Q1B: Does patient gender predict APACHE IVa score? (SE)

import pandas as pd
import numpy as np
import csv
import scipy as sp
import scipy.stats as stat

In [15]:
pts = pd.read_csv('patient.csv')

In [16]:
apachescores = pd.read_csv('apachePatientResult.csv')

In [17]:
apachescores.head()

,apachepatientresultsid,patientunitstayid,physicianspeciality,physicianinterventioncategory,acutephysiologyscore,apachescore,apacheversion,predictedicumortality,actualicumortality,predictediculos,...,predictedhospitallos,actualhospitallos,preopmi,preopcardiaccath,ptcawithin24h,unabridgedunitlos,unabridgedhosplos,actualventdays,predventdays,unabridgedactualventdays
0,26570,141168,critical care medicine (CCM),Unknown,49,65,IV,0.026988,EXPIRED,3.038388,...,7.546453,2.4972,0,0,0,2.4972,2.4972,NaN,NaN,NaN
1,26571,141168,critical care medicine (CCM),Unknown,49,65,IVa,0.028889,EXPIRED,3.091127,...,6.628720,2.4972,0,0,0,2.4972,2.4972,NaN,NaN,NaN
2,53135,141194,critical care medicine (CCM),Unknown,57,70,IV,0.037888,ALIVE,4.620982,...,13.338449,9.2167,0,0,0,3.3423,9.2167,NaN,NaN,NaN
3,53136,141194,critical care medicine (CCM),Unknown,57,70,IVa,0.046448,ALIVE,4.167129,...,12.978228,9.2167,0,0,0,3.3423,9.2167,NaN,NaN,NaN
4,8,141203,hospitalist,I,73,90,IVa,0.291609,ALIVE,8.670299,...,16.319389,3.7493,0,0,0,1.2979,3.7493,2.0,5.738093,2.0


In [18]:
# in case per patient analysis needed
unipts = pts.drop_duplicates('uniquepid')

In [19]:
# for now continuing with analysis per trauma admission (i.e. if patient X comes in twice for trauma, n = 2 for that patient)

q = 'trauma'

mpts = pts[pts['gender'] == 'Male']
fpts = pts[pts['gender'] == 'Female']

mepts = mpts[mpts['unitadmitsource'] == 'Emergency Department']
fepts = fpts[fpts['unitadmitsource'] == 'Emergency Department']

# male and female trauma ED-ICU admissions
msepts = mepts[mepts['apacheadmissiondx'].str.contains(q).fillna(False)]
fsepts = fepts[fepts['apacheadmissiondx'].str.contains(q).fillna(False)]

In [20]:
msepts.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
14,141264,128998,Male,19,Caucasian,59,91,Head only trauma,188.0,02:24:29,...,Emergency Department,1,admit,NaN,80.1,09:02:00,292,Step-Down Unit (SDU),Alive,002-47110
488,144519,131512,Male,56,Caucasian,68,103,Head only trauma,180.3,23:07:00,...,Emergency Department,1,admit,99.80,105.4,20:42:00,1293,Floor,Alive,002-33534
511,144689,131641,Male,57,Caucasian,73,97,Extremity only trauma,170.2,23:15:00,...,Emergency Department,1,admit,49.89,55.6,21:06:00,4190,Floor,Alive,002-38176
665,145604,132349,Male,85,Hispanic,73,100,Head only trauma,152.4,14:48:00,...,Emergency Department,1,admit,63.50,54.0,04:43:00,2273,Floor,Alive,002-46870
747,146127,132760,Male,51,African American,73,99,Head/face trauma,177.8,19:31:00,...,Emergency Department,1,admit,79.30,73.8,21:39:00,1559,Step-Down Unit (SDU),Alive,002-44708


In [88]:
fapachescores.head()

,apachepatientresultsid,patientunitstayid,physicianspeciality,physicianinterventioncategory,acutephysiologyscore,apachescore,apacheversion,predictedicumortality,actualicumortality,predictediculos,...,predictedhospitallos,actualhospitallos,preopmi,preopcardiaccath,ptcawithin24h,unabridgedunitlos,unabridgedhosplos,actualventdays,predventdays,unabridgedactualventdays
729,132982,144770,critical care medicine (CCM),Unknown,57,81,IV,0.238317,ALIVE,6.065513,...,14.350764,28.7444,0,0,0,27.8597,28.7444,2.0,3.379165,2.0
1058,85239,146350,critical care medicine (CCM),Unknown,55,60,IVa,0.063915,ALIVE,5.594285,...,-1.000000,20.1264,0,0,0,2.5548,20.1264,NaN,NaN,NaN
1113,58749,146564,critical care medicine (CCM),Unknown,49,66,IVa,0.078707,ALIVE,4.437731,...,8.527262,48.5764,0,0,0,8.6895,48.5764,3.0,1.705564,3.0
1274,64083,147390,surgery-cardiac,I,29,29,IV,0.003014,ALIVE,0.170000,...,3.977961,7.0299,0,0,0,3.2756,7.0299,NaN,NaN,NaN
1301,101318,147511,critical care medicine (CCM),Unknown,58,63,IV,0.037273,ALIVE,5.582192,...,12.373300,8.1389,0,0,0,2.0243,8.1389,2.0,3.912112,2.0


In [93]:
mapachescores['apachescore'].iloc[3]

51

In [94]:
mreind = msepts.reset_index()
freind = fsepts.reset_index()

mapachescores = apachescores.iloc[msepts.index.tolist()]
fapachescores = apachescores.iloc[fsepts.index.tolist()]

ma4 = []
fa4 = []
ma4a = []
fa4a = []

for i in range(len(mapachescores)):
    if mapachescores['apacheversion'].iloc[i] == 'IV':
        ma4.append(mapachescores['apachescore'].iloc[i])
    if mapachescores['apacheversion'].iloc[i] == 'IVa':
        ma4a.append(mapachescores['apachescore'].iloc[i])
                

for i in range(len(fapachescores)):
    if fapachescores['apacheversion'].iloc[i] == 'IV':
        fa4.append(fapachescores['apachescore'].iloc[i])
    if fapachescores['apacheversion'].iloc[i] == 'IVa':
        fa4a.append(fapachescores['apachescore'].iloc[i])

# Q tech, is there a way to do this pythonically with indexing rather than for loops?

In [104]:
len(fa4)


913

In [105]:
print np.mean(ma4)
print np.mean(fa4)
print np.mean(ma4a)
print np.mean(fa4a)

54.54164057608015
54.721796276013144
54.1940203562341
53.48315911730546


In [106]:
print stat.ttest_ind(ma4, fa4)

print stat.ttest_ind(ma4a, fa4a)

Ttest_indResult(statistic=-0.17146115777504814, pvalue=0.8638750020743301)
Ttest_indResult(statistic=0.6596664895856557, pvalue=0.5095303633012185)
